In [74]:
#Author: Grant Parker
#Functions/packages/code taken, adapated, and modified from https://github.com/microsoft/nni/blob/36ba04c94e51eaa2a88b9e6e0a4077f3a9f93004/examples/compression/pruning/scheduled_pruning.py
#NNI packages
import nni
from nni.compression import TorchEvaluator
from nni.compression.pruning import TaylorPruner, AGPPruner
from nni.compression.utils import auto_set_denpendency_group_ids
from nni.compression.speedup import ModelSpeedup


#pytorch packages 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

#tensor packages 
from tensorboardX import SummaryWriter
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

from models import LeNet_5
from datasets import get_mnist_dataloaders

import models
import datasets
#import modelsNNI

import time

#imports from Models
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader
import inspect

#importing Resnet 
import torchvision.models as modelsRes 

In [75]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [76]:

#functions
def train(model: torch.nn.Module, optimizer: torch.optim.Optimizer, training_step,
          lr_scheduler: _LRScheduler, max_steps: int, max_epochs: int):
    assert max_epochs is not None or max_steps is not None
    #train_loader, test_loader = prepare_dataloader()
    #grant addittion ---
    batch_size = 100
    train_loader, test_loader = datasets.get_mnist_dataloaders(batch_size,batch_size)

    #----
    max_steps = max_steps if max_steps else max_epochs * len(train_loader)
    max_epochs = max_steps // len(train_loader) + (0 if max_steps % len(train_loader) == 0 else 1)
    count_steps = 0

    model.train()
    for epoch in range(max_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = training_step((data, target), model)
            loss.backward()
            optimizer.step()
            count_steps += 1
            if count_steps >= max_steps:
                acc = evaluate(model, test_loader)
                print(f'[Training Epoch {epoch} / Step {count_steps}] Final Acc: {acc}%')
                return
        acc = evaluate(model, test_loader)
        print(f'[Training Epoch {epoch} / Step {count_steps}] Final Acc: {acc}%')

def evaluate(model: torch.nn.Module, test_loader):
    model.eval()
    correct = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return 100 * correct / len(test_loader.dataset)


def training_step(batch, model: torch.nn.Module):
    output = model(batch[0])
    loss = F.cross_entropy(output, batch[1])
    return loss

def prepare_optimizer(model: torch.nn.Module):
    optimize_params = [param for param in model.parameters() if param.requires_grad == True]
    optimizer = nni.trace(Adam)(optimize_params, lr=0.01)
    return optimizer

In [77]:
class LeNet_300_100(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

In [78]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if __name__ == '__main__':
    print("start")
    #grants additions 

    model = LeNet_300_100().to(device)
    #model = modelsRes.resnet18().to(device)
    #optimizer = optim.NAdam(model.parameters()) -- did not use this 
    optimizer = prepare_optimizer(model)
    batch_size = 100
    train(model, optimizer, training_step, lr_scheduler=None, max_steps=None, max_epochs=50)

    train_loader, test_loader = datasets.get_mnist_dataloaders(batch_size, batch_size)
    print('Original model paramater number: ', sum([param.numel() for param in model.parameters()]))
    print('Original model after 10 epochs finetuning acc: ', evaluate(model, test_loader), '%')

    

start
[Training Epoch 0 / Step 600] Final Acc: 95.16%
[Training Epoch 1 / Step 1200] Final Acc: 96.57%
[Training Epoch 2 / Step 1800] Final Acc: 96.83%
[Training Epoch 3 / Step 2400] Final Acc: 96.87%
[Training Epoch 4 / Step 3000] Final Acc: 96.46%
[Training Epoch 5 / Step 3600] Final Acc: 97.01%
[Training Epoch 6 / Step 4200] Final Acc: 96.88%
[Training Epoch 7 / Step 4800] Final Acc: 97.16%
[Training Epoch 8 / Step 5400] Final Acc: 97.24%
[Training Epoch 9 / Step 6000] Final Acc: 97.36%
Original model paramater number:  266610
Original model after 10 epochs finetuning acc:  97.36 %


In [79]:
target_sparsity = .99
config_list = [{
    'op_names': ['fc1', 'fc2'],
    'sparse_ratio':target_sparsity
}]
#config_list = [{
    #'op_names': ['conv1', 'conv2', 'fc3', 'fc4'],
    #'sparse_ratio': target_sparsity,
    #'global_group_id': 'fourLayers'
#}]
""" config_list = [{
    'op_names': ['conv1'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['conv2'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc3'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc4'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}] """
""" config_list = [{
    'op_names': ['conv1', 'conv2'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc3','fc4'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}] """


dummy_input = torch.rand(6,1,28,28)
config_list = auto_set_denpendency_group_ids(model, config_list, dummy_input)
optimizer = prepare_optimizer(model)
evaluator = TorchEvaluator(train, optimizer, training_step)

sub_pruner = TaylorPruner(model,config_list, evaluator, training_steps=100)
scheduled_pruner = AGPPruner(sub_pruner, interval_steps=100, total_times=10)

_, masks = scheduled_pruner.compress(max_steps=100 * 10, max_epochs=None)
scheduled_pruner.unwrap_model()

model = ModelSpeedup(model, dummy_input, masks).speedup_model()

[Training Epoch 0 / Step 600] Final Acc: 63.17%
[Training Epoch 1 / Step 1000] Final Acc: 23.72%
[2023-12-06 21:02:04] Start to speedup the model...
[2023-12-06 21:02:04] Resolve the mask conflict before mask propagate...
[2023-12-06 21:02:04] WARNING: no multi-dimension masks found.
0 Filter
[2023-12-06 21:02:04] WARNING: no multi-dimension masks found.
[2023-12-06 21:02:04] Infer module masks...
[2023-12-06 21:02:04] Propagate original variables
[2023-12-06 21:02:04] Propagate variables for placeholder: x, output mask:  0.0000 
[2023-12-06 21:02:04] Propagate variables for call_method: view, output mask:  0.0000 
[2023-12-06 21:02:04] Propagate variables for call_module: fc1, weight:  0.9900 bias:  0.9900 , output mask:  0.0000 
[2023-12-06 21:02:04] Propagate variables for call_function: relu, output mask:  0.0000 
[2023-12-06 21:02:04] Propagate variables for call_module: fc2, weight:  0.9900 bias:  0.9900 , output mask:  0.0000 
[2023-12-06 21:02:04] Propagate variables for call_f

In [80]:
print('Pruned model paramater number: ', sum([param.numel() for param in model.parameters()]))
print(model)

forward_source = inspect.getsource(model.forward)
print(forward_source)

Pruned model paramater number:  2379
LeNet_300_100(
  (fc1): Linear(in_features=784, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
  (fc3): Linear(in_features=1, out_features=10, bias=True)
)
    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)



In [81]:
import math

class LeNet_300_100Pruned(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(784, int(math.ceil(300-(300*target_sparsity))))
        self.fc2 = nn.Linear(int(math.ceil(300-(300*target_sparsity))), int(math.ceil(100-(100*target_sparsity))))
        self.fc3 = nn.Linear(int(math.ceil(100-(100*target_sparsity))), 10)

    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)


    

In [82]:
modelPruned = model

print(modelPruned)

forward_source = inspect.getsource(modelPruned.forward)
print(forward_source)

LeNet_300_100(
  (fc1): Linear(in_features=784, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
  (fc3): Linear(in_features=1, out_features=10, bias=True)
)
    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)



In [83]:
optimizer = prepare_optimizer(modelPruned)
train(modelPruned, optimizer, training_step, lr_scheduler=None, max_steps=None, max_epochs=50)

[Training Epoch 0 / Step 600] Final Acc: 44.58%
[Training Epoch 1 / Step 1200] Final Acc: 45.59%
[Training Epoch 2 / Step 1800] Final Acc: 46.8%
[Training Epoch 3 / Step 2400] Final Acc: 45.51%
[Training Epoch 4 / Step 3000] Final Acc: 46.97%
[Training Epoch 5 / Step 3600] Final Acc: 47.48%
[Training Epoch 6 / Step 4200] Final Acc: 47.02%
[Training Epoch 7 / Step 4800] Final Acc: 47.28%
[Training Epoch 8 / Step 5400] Final Acc: 48.12%
[Training Epoch 9 / Step 6000] Final Acc: 49.71%


In [84]:
print('Pruned model after 10 epochs finetuning acc: ', evaluate(modelPruned, test_loader), '%')

Pruned model after 10 epochs finetuning acc:  49.71 %
